# Byte-Pair Encoding Tokenization
Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts, and then used by OpenAI for tokenization when pretraining the GPT model. It’s used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa. In this exercise, you will be implementing BPE based on a simple text corpus, namely the text of the Constitution of New York (1777).

## Part 1: Text Tokenization
The first task is to implement a `tokenize` function that accepts a string and a vocabulary and turns the input string into a list of tokens. For example:

```python
>>> tokenize("hello world", [" wo", "rld", "he", "llo"])
['he', 'llo', ' wo', 'rld']
>>> tokenize("hehehe", [" wo", "rld", "he", "llo"])
['he', 'he', 'he']
```

If the provided input string cannot be parsed into the provided vocabulary of tokens, you can raise an exception.

**HINT**: Use `s.startswith(prefix)`

In [ ]:
def tokenize(chunk: str, vocab: list[str]) -> list[str]:
  tokens = []
  vocab = sorted(vocab, key = len, reverse = True)
  while chunk:
    match = None
    for token in vocab:
      if (chunk.startswith(token)):
        match = token
        break
    if (match is None):
      raise ValueError(f'No tokens for {chunk}')
    tokens.append(match)
    chunk = chunk[len(match):]
  return tokens
tokenize("hello world", [" wo", "rld", "he", "llo", " world"])


['he', 'llo', ' world']

> **Note on Optimal Tokenization**:
> As the vocabulary grows, it's important to prefer longer tokens where you have a choice, for example:
> ```python
> >>> tokenize("hello world", [" wo", "rld", "he", "llo", " world"])
> ['he', 'llo', ' world']
> ```
> If your initial implementation does not account for this you can revisit before proceeding.



## Part 2: Vocabulary Construction
The second task is to derive a vocabulary of a specific size based on the provided corpus. The algorithm can be broken down into the following steps:

1. Operate on the chunks defined below as a corpus.
2. Assemble a character-level vocabulary—i.e., a vocabulary of tokens in which all tokens are of length 1—based on all characters that appear throughout the text chunks.
3. Merge the pair of tokens that occurs most frequently throughout the chunks into a new, longer token.

    a. First, implement a `to_pairs` function in terms of tokenize that accepts a text chunk and a vocabulary and returns the list of pairs that occur in the chunk

    b. Then, implement a `next_most_frequent_pair` function that accepts the full list of text chunks and the current vocabulary to determine the next pair of tokens that should be merged


Step 3 can be repeated until the desired vocabulary size is reached. For this exercise, build a vocabulary of size 256.


In [ ]:
%pip install html2text

In [ ]:
import requests
import html2text
h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_images = True

# Load full text of document and split into chunks
full_text = h.handle(requests.get('https://avalon.law.yale.edu/18th_century/ny01.asp').text)
chunks = full_text.split('\n\n')


In [ ]:
def get_base_vocab(text):
  # get all the unique characters from the text
  base_vocab = list(set(text))
  print(len(base_vocab))
  return base_vocab
base_vocab = get_base_vocab(full_text)

80


In [ ]:
def to_pairs(chunk: str, vocab):
  pairs = []
  tokens = tokenize(chunk, vocab)
  for i in range(len(tokens) - 1):
    pairs.append((tokens[i], tokens[i + 1]))
  return pairs

In [ ]:
print(to_pairs(chunks[0], base_vocab))

[(' ', ' '), (' ', '*'), ('*', ' '), (' ', '|'), ('|', ' '), (' ', ' '), (' ', ' '), (' ', '\n'), ('\n', '-'), ('-', '-'), ('-', '-'), ('-', '|'), ('|', '-'), ('-', '-'), ('-', '-'), ('-', ' '), (' ', ' ')]


In [ ]:
def next_most_frequent_pair(chunks, vocab):
  pairs_count = {}
  for chunk in chunks:
    pairs = to_pairs(chunk, vocab)
    for pair in pairs:
      pairs_count[pair] = pairs_count.get(pair, 0) + 1
  return max(pairs_count, key = pairs_count.get)

In [ ]:
next_most_frequent_pair(chunks, base_vocab)

('e', ' ')

In [ ]:
from tqdm import tqdm

def build_vocab(chunks, target_size = 256):
  vocab = base_vocab
  while (len(vocab) < target_size):
    most_frequent_pair = next_most_frequent_pair(chunks, vocab)
    if most_frequent_pair is None:
      break
    new_token = most_frequent_pair[0] + most_frequent_pair[1]
    vocab.append(new_token)
  return vocab

In [ ]:
print(build_vocab(chunks))

['S', '0', '2', 'O', 'C', 'V', 'n', '|', '3', '9', "'", 'u', '(', 'Â', '£', '8', 'i', 'r', 'b', '#', 'l', ')', 'k', '4', 'E', '_', 'g', 'A', 'X', '6', 'F', 'a', 'j', '~', ',', 'N', ';', 'P', 'U', 'Y', 'h', 'M', 's', 'W', ':', '.', 'R', 'p', '-', 'T', 'H', '>', '*', 'K', '"', 'I', 'v', 't', 'L', 'o', '5', 'f', 'q', ' ', '1', 'x', 'w', 'e', 'm', 'd', 'J', 'G', 'Q', 'D', 'y', 'z', 'c', '\n', '7', 'B', 'e ', 'th', ' th', ' a', ' o', 'er', 'in', ' the ', 'd ', 'on', 'at', 'en', 'es', ' of', ', ', 'or', 'an', 's ', 'al', 'ti', 'to', 're', 'of', 'ed ', 't ', 'y ', 'and ', 'all', 'sh', 'the ', 'ou', 'ec', 'ar', ' b', 'is', 'shall', 'ch', 'e, ', 'con', 'it', 'for', 'ver', ' an', 'is ', 'of ', 'tat', '. ', 'e\n', 'su', 'res', 'ol', 'tion', 'to ', 's, ', 'el', 'ing', 'ic', 'em', 'wh', 'as ', 'ed', 'of the ', 'ro', 'Stat', 'as', 'r ', 'by ', 'ent ', 'ion', 'go', 'ter', 'sa', 'at ', 'ther', 'cou', 'ur', 'e of ', 'Th', 'pe', 'in ', 'bl', 'ent', 'be ', 'ac', 'un', ' and ', 'this ', 'il', 'ith', 'id',